# Jupyter Notebook Tutorial 

Roughly, the design idea of ``mdciao`` is that:

* The [CLI](http://proteinformatics.uni-leipzig.de/mdciao/cli_cli/cli_cli.html) offers pre-packaged analysis pipelines that are essentially *one-shot* tools. They are an entry-point for non-experts and do not require any Python scripting knowledge. CLI tools are still highly customizable (check ``mdc_**.py -h`` or ``mdc_examples.py``), but offer **only some** of the ``mdciao``-functionalities.
  
* The [Python API](http://proteinformatics.uni-leipzig.de/mdciao/api/api.html), on the other hand, exposes:
   - **CLI-equivalent** functions via the ``mdciao.cli`` [submodule](http://proteinformatics.uni-leipzig.de/mdciao/cli_cli/cli_cli.html). Here you'll find evertying that the CLI offers, only as regular Python functions. This provides scripting flexibility, with the added value that now input and outputs are *normal* Python objects that can be further manipulated, by ``mdciao`` or any other Python module of your liking.
   - Other **standalone submodules** that the CLI uses *under the hood*, and that the user can access directly for any other scripting purpuse: plotting methods, alignment/sequence methods, nomenclature methods, PDB-methods etc.

<div class="alert alert-info">
    
<b>Note</b> 
    
**THE API IS NOT STABLE YET**, if you are using ``mdciao`` in API mode, we assume you can handle future API changes without much hassle.

</div>

For clarity, this notebook loosely follows the same structure as the [Overview](http://proteinformatics.uni-leipzig.de/mdciao/overview.html) section of the ``mdciao``documentation. Other notebooks will follow soon, explaining basic concepts and/or advanced pipelines.

If you want to run this notebook on your own, please download and extract the data from [here](http://proteinformatics.org/mdciao/mdciao_example.zip) first. You can download it:

* using the browser 
* using the terminal with  
 ```wget http://proteinformatics.org/mdciao/mdciao_example.zip; unzip mdciao_example.zip```
* using  mdciao's own method [mdciao.examples.fetch_example_data](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.examples.fetch_example_data.html#mdciao-examples-fetch-example-data)

In [ ]:
import mdciao, os
if not os.path.exists("mdciao_example"):
    mdciao.examples.fetch_example_data()

## Basic Usage
Now we replicate the CLI command:

```
mdc_neighborhoods.py prot.pdb traj.xtc --residues L394 -nf #nf: don't use fragments
```

but in API mode. We use the method [cli.residue_neighborhoods](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.cli.residue_neighborhoods.html#mdciao.cli.residue_neighborhoods):

In [ ]:
result= mdciao.cli.residue_neighborhoods("L394",
                                         "mdciao_example/traj.xtc", 
                                         "mdciao_example/prot.pdb", 
                                         fragments=[None])

``result`` is a dictionary of dictionaries, with the main result under the key ``neighborhoods``. There, you'll find a dictionary keyed with residue indices and valued with a [ContactGroup](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup) for each residue neighborhood. 

[ContactGroups](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup) are ``mdciao`` classes that allow the further manipulation of contact data, molecular information and much more. Check here to learn more about ``mdciao`` [classes](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/mdciao.contacts.html).

In [ ]:
result["neighborhoods"][353]

## Using Python Objects
Please note that in API mode, inputs can be objects, for example ``mdtraj`` [Trajectories](https://mdtraj.org/1.9.4/api/generated/mdtraj.Trajectory.html). So, before calling the next ``mdciao.cli`` method, we use ``mdtraj`` to load the trajectory from our files:

In [ ]:
import mdtraj as md
traj = md.load("mdciao_example/traj.xtc", top="mdciao_example/prot.pdb")
traj

And we repeat the above command using the ``traj`` object. Please note that we're also using the ``no_disk`` option so that no files are written to disk, in case we're only interested in working in memory.

In [ ]:
result = mdciao.cli.residue_neighborhoods("L394",
                                          traj,
                                          fragments=[None],
                                          no_disk=True)

Now, the more elaborated CLI-command:

```
mdc_neighborhoods.py prot.pdb traj.xtc -r L394 --GPCR adrb2_human --CGN 3SN6 -ni -at #ni: not interactive, at: show atom-types
```

We keep the ``no_disk`` option to avoid writing to disk, but you can change this if you want. **Please note** that some options **do not carry** exactly the same names as their CLI equivalents. E.g. ``ni`` in the CLI (= *don't be interactive*) is now ``accept_guess`` in the API. These differences are needed for compatiblity with other methods, but might get unified in the future. 

In [ ]:
result = mdciao.cli.residue_neighborhoods("L394",
                                          traj,
                                          GPCR_uniprot="adrb2_human",
                                          CGN_PDB="3SN6",
                                          accept_guess=True,
                                          plot_atomtypes=True,
                                          no_disk=True)

## Consensus Nomenclature: GPCR and/or CGN
Above, we declared our intention to use GPCR consensus nomenclature and Common G-alpha Numbering (CGN) by passing the descriptor strings ``GPCR_uniprot="adrb2_human"`` or ``CGN_PDB="3SN6"``, respectively, to contact the online databases, in particular the  

* [GPCRdb](https://gpcrdb.org/), Kooistra et al, (2021) GPCRdb in 2021: Integrating GPCR sequence, structure and function, Nucleic Acids Research 49, D335--D343, https://doi.org/10.1093/nar/gkaa1080

For that retrieval and handling of these labels, we use the [module](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/mdciao.nomenclature.html) ``mdciao.nomenclature``, which offers [two main classes](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/mdciao.nomenclature.html#classes) and other helper function to deal with the nomenclature. An overview of the relevant references is contained in [the module's documentation](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/mdciao.nomenclature.html) as well as in the documentation of each class, e.g. [here](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.LabelerGPCR.html#mdciao.nomenclature.LabelerGPCR).

Additionally, you can use [mdciao.nomenclature.references](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.references.html) to print all relevant nomenclature references:

In [ ]:
mdciao.nomenclature.references()

The nomenclature classes produce standalone objects, and can do much more than just be inputs to `mdciao.cli` methods. As with any Python class, you can learn a lot about its methods and attributes by using the [tab autocompletion feature of IPython](https://ipython.org/ipython-doc/dev/interactive/tutorial.html#tab-completion). Or you can check [here](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/mdciao.nomenclature.html) for more ``mdciao`` docs. 

Since we'll be using these labels more than once in the notebook, instead of using the network each time, we can have them as Python objects in memory. Alternatively, it's possible to save the labeling data locally after the first database query. This allows for inspection and re-use of the retrieved data outside the notebook (in a spreadsheet, for example).

In [ ]:
from mdciao import nomenclature
GPCR = nomenclature.LabelerGPCR("adrb2_human", 
                            #write_to_disk=True 
                           )
CGN = nomenclature.LabelerCGN("3SN6", 
                             # write_to_disk=True
                             )

## Residue Selection
Now, we can play around with residue selection, replicating the CLI-command:

```
mdc_residues.py GLU*,P0G,380-394,G.HN.* prot.pdb --GPCR adrb2_human --CGN 3SN6 -ni
```

Check the docs [here](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.cli.residue_selection.html) to check the output values `res_idxs_list`,` fragments`, and `consensus_maps`, although most of out useful output is written out.

Please note that we're now using ``mdciao.nomenclature`` classes directly as inputs (``GPCR`` and ``CGN``), speeding up the method by avoiding queries over the network.

In [ ]:
res_idxs_list, fragments, consensus_maps = mdciao.cli.residue_selection(
    "GLU*,P0G,380-394,G.HN.*",
    traj,
    GPCR_uniprot=GPCR,
    CGN_PDB=CGN,
    accept_guess=True)

## PDB Queries
Now we grab a structure directly from the PDB, replicating the CLI command:

```
mdc_pdb.py 3SN6 -o 3SN6.gro
```

by using `mdciao.cli.pdb`. Check [here](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.cli.pdb.html#mdciao.cli.pdb) or use the inline docstring for more info. Please note that we're not storing the retrived structure on disk, but rather having it in memory as an ``mdtraj.Trajectory``:

In [ ]:
xray3SN6= mdciao.cli.pdb("3SN6")
xray3SN6

Now we can use the [awesome nglviewer](https://github.com/nglviewer/nglview/) to 3D-visualize the freshly grabbed structure inside the notebook.

We need to import the module first, which needs to be installed in your Python environment. If you don't we recommend you [install](https://github.com/nglviewer/nglview/#installation) it via pip:

```
pip install nglview
jupyter-nbextension enable nglview --py --sys-prefix
```

If you don't feel like installing now, you can continue use the notebook. 

In [ ]:
try:
    import nglview
    iwd = nglview.show_mdtraj(xray3SN6)
except ImportError:
    iwd = None
iwd


## Fragmentation Heuristics
Now we go to fragmentation heuristics, replicating the CLI command:

```
mdc_fragments.py 3SN6.gro
```

but with the object ``xray3SN6`` (the ``.gro``-file comes below) and by using the ``cli.fragments`` method. Check [here](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.cli.fragment_overview.html#mdciao.cli.fragment_overview) or the inline docstring for more info. Also note that ``cli.fragments`` simply wraps around [mdciao.fragments.get_fragments](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.fragments.get_fragments.html), and you can use that method (or others in [mdciao.fragments](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/mdciao.fragments.html)) directly.

In [ ]:
frags= mdciao.cli.fragment_overview(xray3SN6.top)

This call iterates through all available heuristics on the ``mdtraj``[Topology](https://mdtraj.org/1.9.4/api/generated/mdtraj.Topology.html), arriving at different definitions of molecular fragments. They are all returned as a dictionary:

In [ ]:
frags.keys()

Please note that since ``xray3SN6`` comes from the PDB directly, it contains chain descriptors, s.t. the method ``chains`` (first one) can simply list the chain information encoded into the PDB, which you can check [here](https://www.rcsb.org/sequence/3SN6):

```
Auto-detected fragments with method 'chains'
fragment      0 with    349 AAs     THR9 (     0) -   LEU394 (348   ) (0)  resSeq jumps
fragment      1 with    340 AAs     GLN1 (   349) -   ASN340 (688   ) (1) 
fragment      2 with     58 AAs     ASN5 (   689) -    ARG62 (746   ) (2) 
fragment      3 with    443 AAs  ASN1002 (   747) -   CYS341 (1189  ) (3)  resSeq jumps
fragment      4 with    128 AAs     GLN1 (  1190) -   SER128 (1317  ) (4) 
fragment      5 with      1 AAs  P0G1601 (  1318) -  P0G1601 (1318  ) (5)       5 with    1 AAs  P0G1601           (1318) -  P0G1601           (1318) (5) 
```

These fragments are:

0. G-protein $\alpha$ sub-unit
1. G-protein $\beta$ sub-unit
2. G-protein $\gamma$ sub-unit
3. $\beta_2$ adrenergic receptor, together with the bacteriophage T4 lysozyme as N-terminus
4. VHH nanobody
5. Ligand P0G

However, we loose that chain information if we store the structure as ``.gro``, which doesn't encode for chains (i.e., the entire topology is put into a single chain).

In [ ]:
from tempfile import NamedTemporaryFile
with NamedTemporaryFile(suffix=".gro") as tmpgro:
    xray3SN6.save(tmpgro.name)
    xray3SN6gro = md.load(tmpgro.name)
mdciao.cli.fragment_overview(xray3SN6gro.top, methods=["chains", "lig_resSeq+"]);

The ``lig_resSeq+`` (the current default) has attempted to recover some meaningful fragments, closely resembling the original chains:

0. G-protein $\alpha$ sub-unit
1. G-protein $\beta$ sub-unit
2. G-protein $\gamma$ sub-unit
3. Bacteriophage T4 lysozyme as N-terminus
4. $\beta_2$ adrenergic receptor
5. VHH nanobody
6. Ligand P0G

The former fragment 3 (4TL-$\beta_2$AR) chain has been broken up into T4L and $\beta_2$AR.

## Interfaces
Now we move to a more elaborate command:

```
mdc_interface.py prot.pdb traj.xtc -fg1 0 -fg2 3 --GPCR adrb2_human --CGN 3SN6 -t "3SN6 beta2AR-Galpha interface" -ni
```

and replicate it using ``cli.interface``. Check the docs [here](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.cli.interface.html#mdciao.cli.interface) or in the method's docstring. 

Additionally, we now have two other notebooks explicitly devoted to the representation of interfaces:

* [Bar Plots](https://proteinformatics.uni-leipzig.de/mdciao/notebooks/Comparing_CGs_Bars.html)
* [FlarePlots](https://proteinformatics.uni-leipzig.de/mdciao/notebooks/Comparing_CGs_Flares.html)

In [ ]:
mdciao.cli.interface(traj,
                     frag_idxs_group_1=[0],
                     frag_idxs_group_2=[3],
                     GPCR_uniprot=GPCR,
                     CGN_PDB=CGN,
                     title="3SN6 beta2AR-Galpha interface",
                     accept_guess=True,
                     plot_timedep=False,
                     no_disk=True)

## Sites
Now we use a different approach. Instead of letting ``mdciao`` discover contacts automatically, we list them beforehand as ``site`` dictionaries, and feed this dictionaries to directly to the [method](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.cli.sites.html) ``cli.sites``. The CLI command we're replicating is:

```
mdc_sites.py prot.pdb traj.xtc --site tip.json -at -nf -sa #sa: short AA-names
```

However, in the API-spirit, we're not even using a file on disk to define the ``site``, but create it on the fly as a Python dictionary:

In [ ]:
my_site = {
    "name":"interface small",
    "pairs": {"AAresSeq": [
        "L394-K270",
        "D381-Q229",
        "Q384-Q229",
        "R385-Q229",
        "D381-K232",
        "Q384-I135"
        ]}}

In [ ]:
sites = mdciao.cli.sites([my_site],
                         traj,
                         no_disk=True,
                         plot_atomtypes=True,
                         fragments=[None],
                         short_AA_names=True)

The return value ``sites`` is a dictionary keyed with the site names (``interface small`` in this case) and valued with ``mdciao's`` [ContactGroup](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup)-objects.

In [ ]:
sites

## Contact Groups
The [ContactGroup](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup) class is at the center of ``mdciao`` and offers extensive of manipulation through it's methods. A helpful analogy would be that, what the [Trajectory](https://mdtraj.org/1.9.4/api/generated/mdtraj.Trajectory.html) is to ``mdtraj``, the [ContactGroup](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup) is to ``mdciao``. Both classes:
 
 * store a lot of organized information for further use
 * have attributes and methods that can be used standalone
 * can themselves be the input for other methods (of ``mdtraj`` and ``mdciao``, respectively). 
 * are rarely created from scratch, but rather generated by the module itself.

The best way to learn about the [ContactGroup](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup) is to inspect it with the autocomplete feature if IPython and check the informative names of the attributes and methods.

If you're in a hurry, ``mdciao`` offers a quick way to generate a [ContactGroup](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup) to play around with and investigate it's methods and attributes:

In [ ]:
from mdciao import examples
CG = examples.ContactGroupL394()
CG

However, instead of using ``CG`` now, we go back to object ``sites`` that resulted from using ``cli.sites`` above. The returned ``sites``-object is a dictionary keyed with site names (you can compute different sites simultaneously) and valued with [ContactGroups](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup). In our case (check above) we called it it *interface small*

In [ ]:
mysite = sites["interface small"]

### Frequencies as Bars
We use the class's method [plot_freqs_as_bars](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup.plot_freqs_as_bars) to produce the now familiar neighborhood plots:

In [ ]:
mysite.plot_freqs_as_bars(3.5, 
                          shorten_AAs=True, 
                          defrag="@", 
                          atom_types=True);

### Frequencies as Distributions
It is also very useful to inspect the residue-residue distances of any [ContactGroup](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup) by looking at their **overall distributions** instead of their frequencies, since the hard cutoffs can sometimes hide part of the story:

In [ ]:
jax = mysite.plot_distance_distributions(bins=15,
                                         defrag="@",
                                         ctc_cutoff_Ang=3.5
                                        )

Please note that, because the example dataset is quite small (280 frames and 2.8 ns) and we are simply histogramming (=counting), the curves aren't very smooth. Histograms of real data will look better.

### Frequencies as Violins
Other ways of looking at distance-data as distributions is to use [violin plots](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup.plot_violins), which uses a density estimator (check the ``bw_method``-parameter [here](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.violinplot.html))  to generate smooth densities and plot them vertically. This is somehow in-between the histogram plot and the frequency-bar plot:

In [ ]:
jax = mysite.plot_violins(defrag="@",
                          ctc_cutoff_Ang=3.5,
                          color="tab10", 
                         )

<div class="alert alert-info">
    
<b>Note</b> 
    
In principle,  we could also use a density estimator in ``plot_distance_distributions``to make them look smooth, but we have decided to leave them as.
</div>

## Comparisons Between Contact Groups
Finally, we replicate the CLI comand

```
mdc_compare.py 3SN6.X.ARG131@4.0_Ang.dat 3SN6.MD.ARG131@4.0_Ang.dat -k Xray,MD -t "3SN6 cutoff 4AA" -a R131
```

in API mode. This looks different because most of the inputs will now be Python objects in memory.

First, we create the Xray and the MD ContactGroups separately:

In [ ]:
R131_Xray = mdciao.cli.residue_neighborhoods("R131", xray3SN6, 
                                             ctc_cutoff_Ang=4,
                                             no_disk=True,
                                             GPCR_uniprot=GPCR,
                                             figures=False,
                                             CGN_PDB=CGN,
                                             accept_guess=True)["neighborhoods"];

In [ ]:
R131_MD = mdciao.cli.residue_neighborhoods("R131",traj, 
                                           ctc_cutoff_Ang=4,
                                           no_disk=True,
                                           GPCR_uniprot=GPCR,
                                           figures=False,
                                           CGN_PDB=CGN,
                                           accept_guess=True)["neighborhoods"];

Please note that, because the molecular topologies differ, the residue ``R131`` is has different indices in each topology, namely 1007 in the X-ray crystal, 861 in the MD simulation:

In [ ]:
R131_Xray, R131_MD

That will frequently be the case when comparing different proteins of the same family, or topologies where not all sub-units have been modelled etc or antying that produces a shift in these indices. 

``mdciao`` *understands* ``R131`` automatically and doesn't ask more questions, as long as there's an obvious ``R131`` candidate. Otherwise the user will be prompted for disambiguation. 

In this case, now we create a dictionary of ContactGroups that represent the R131 in both topologies:

In [ ]:
R131 = {
    "Xray": R131_Xray[1007],
    "MD"  : R131_MD[861]
}
#np.save("R131.npz",R131)

Now we can just pass this dictionary to ``cli.compare`` to see their contact frequencies. This module is pretty flexible on inputs and outputs, check the [documentation](http://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.cli.compare.html#mdciao.cli.compare) to learn more: 

In [ ]:
mdciao.cli.compare(R131, ctc_cutoff_Ang=4, defrag=None, anchor="R131");